In [1]:
import pandas as pd
import sqlite3
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from sklearn.decomposition import PCA
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm
import time

/Users/mingcao/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
train_data = pd.read_csv('resampled_training.csv', sep='\t')
test_x = pd.read_csv('test_x.csv', sep='\t')
test_y = pd.read_csv('test_y.csv', sep='\t')

In [3]:
test_x.drop('Unnamed: 0',axis=1,inplace=True)
test_y.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
y_test = test_y['level_cor']

In [5]:
train_data.columns.tolist()

['Unnamed: 0',
 'X..1.year',
 'X0.years',
 'X1.year',
 'X10..years',
 'X2.years',
 'X3.years',
 'X4.years',
 'X5.years',
 'X6.years',
 'X7.years',
 'X8.years',
 'X9.years',
 'MORTGAGE.1',
 'RENT.1',
 'ACTIVE',
 'BROKEN',
 'COMPLETE',
 'No',
 'AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY',
 'pc_x_1',
 'pc_x_2',
 'pc_x_3',
 'pc_x_4',
 'pc_x_5',
 'pc_x_6',
 'pc_x_7',
 'pc_x_8',
 'pc_x_9',
 'pc_x_10',
 'pc_x_11',
 'pc_x_12',
 'pc_x_13',
 'pc_x_14',
 'pc_x_15',
 'pc_x_16',
 'pc_x_17',
 'pc_x_18',
 'pc_x_19',
 'pc_x_20',
 'pc_x_21',
 'pc_x_22',
 'pc_x_23',
 'pc_x_24',
 'pc_x_25',
 'pc_x_26',
 'pc_x_27',
 'pc_x_28',
 'pc_x_29',
 'pc_x_30',
 'pc_x_31',
 'pc_x_32',
 'pc_x_33',
 'pc_x_34',
 'p

In [6]:
x_cols = ['X..1.year',
 'X0.years',
 'X1.year',
 'X10..years',
 'X2.years',
 'X3.years',
 'X4.years',
 'X5.years',
 'X6.years',
 'X7.years',
 'X8.years',
 'X9.years',
 'MORTGAGE.1',
 'RENT.1',
 'ACTIVE',
 'BROKEN',
 'COMPLETE',
 'No',
 'AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY',
 'pc_x_1',
 'pc_x_2',
 'pc_x_3',
 'pc_x_4',
 'pc_x_5',
 'pc_x_6',
 'pc_x_7',
 'pc_x_8',
 'pc_x_9',
 'pc_x_10',
 'pc_x_11',
 'pc_x_12',
 'pc_x_13',
 'pc_x_14',
 'pc_x_15',
 'pc_x_16',
 'pc_x_17',
 'pc_x_18',
 'pc_x_19',
 'pc_x_20',
 'pc_x_21',
 'pc_x_22',
 'pc_x_23',
 'pc_x_24',
 'pc_x_25',
 'pc_x_26',
 'pc_x_27',
 'pc_x_28',
 'pc_x_29',
 'pc_x_30',
 'pc_x_31',
 'pc_x_32',
 'pc_x_33',
 'pc_x_34',
 'pc_x_35',
 'pc_x_36',
 'pc_x_37']

In [7]:
X_train = train_data[x_cols]
y_train = train_data['y_class']

In [8]:
X_train.head()

,X..1.year,X0.years,X1.year,X10..years,X2.years,X3.years,X4.years,X5.years,X6.years,X7.years,...,pc_x_28,pc_x_29,pc_x_30,pc_x_31,pc_x_32,pc_x_33,pc_x_34,pc_x_35,pc_x_36,pc_x_37
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.050789,-0.271259,-0.034634,-0.161174,0.212710,0.045789,0.124396,0.297331,0.341977,-0.055169
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.315749,-0.483586,-0.151907,-0.500346,-0.185445,-0.677531,-0.631550,0.035970,0.273273,0.393640
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.392776,0.529390,0.041816,1.151770,-0.327122,-0.280328,0.094972,0.630165,0.171936,0.331221
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.145160,-0.521151,-0.072225,-0.823366,-0.448846,-0.039712,-0.269536,-0.415290,0.613070,0.131326
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.013234,0.742557,0.107804,0.345109,1.494548,-0.652461,0.129092,0.963577,0.989961,0.114925


In [9]:
X_train_scaled = pd.DataFrame(preprocessing.StandardScaler().fit(X_train).transform(X_train),
                             columns=x_cols)

In [10]:
X_train_scaled.head()

,X..1.year,X0.years,X1.year,X10..years,X2.years,X3.years,X4.years,X5.years,X6.years,X7.years,...,pc_x_28,pc_x_29,pc_x_30,pc_x_31,pc_x_32,pc_x_33,pc_x_34,pc_x_35,pc_x_36,pc_x_37
0,-0.264115,-0.246882,-0.238206,-0.64392,-0.279332,3.840565,-0.221891,-0.224074,-0.184335,-0.165416,...,0.070776,-0.328861,-0.045426,-0.202683,0.288362,0.062679,0.171166,0.458669,0.528036,-0.077890
1,-0.264115,-0.246882,4.198050,-0.64392,-0.279332,-0.260378,-0.221891,-0.224074,-0.184335,-0.165416,...,0.412325,-0.597735,-0.206357,-0.656185,-0.234829,-0.932502,-0.899674,0.065034,0.422712,0.660446
2,-0.264115,-0.246882,-0.238206,-0.64392,3.579976,-0.260378,-0.221891,-0.224074,-0.184335,-0.165416,...,0.511616,0.685016,0.059484,1.552833,-0.420996,-0.386010,0.129484,0.959951,0.267362,0.557762
3,-0.264115,-0.246882,-0.238206,-0.64392,-0.279332,-0.260378,4.506714,-0.224074,-0.184335,-0.165416,...,-0.181813,-0.645305,-0.097012,-1.088091,-0.580946,-0.054958,-0.386861,-0.614608,0.943625,0.228914
4,-0.264115,-0.246882,-0.238206,-0.64392,-0.279332,-0.260378,-0.221891,4.462815,-0.184335,-0.165416,...,-1.300810,0.954954,0.150038,0.474260,1.972741,-0.898009,0.177818,1.462102,1.521403,0.201933


In [11]:
# reset classes merging class 3 and 4, and shift class 5 to class 4
y_trainc = y_train.copy()
for i in range(len(y_trainc)):
    if y_trainc[i] == 4:
        y_trainc[i] = 3
    if y_trainc[i] == 5:
        y_trainc[i] = 4

In [12]:
# reset classes merging class 3 and 4, and shift class 5 to class 4
y_testc = y_test.copy()
for i in range(len(y_testc)):
    if y_testc[i] == 4:
        y_testc[i] = 3
    if y_testc[i] == 5:
        y_testc[i] = 4

In [13]:
start = time.time()

In [14]:
clf_rbf = svm.SVC(decision_function_shape='ovo', gamma=0.1, C=1.1, cache_size = 5000, max_iter = 3000, kernel='rbf').fit(X_train, y_trainc) 
rbf_pred = clf_rbf.predict(test_x)

/Users/mingcao/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [15]:
end = time.time()
print(end - start)

1996.6336271762848


In [16]:
print(precision_recall_fscore_support(y_testc, rbf_pred))

(array([0.96409329, 0.96238483, 0.39772572, 0.66472707]), array([0.86448867, 0.98428109, 0.65002239, 0.42109882]), array([0.9115782 , 0.97320981, 0.49349766, 0.51558135]), array([ 29599, 101661,   4466,   5697]))
